# BERTopic - Yahoo! Answers
## ECS271 
## Kay Royo


In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 2.6 MB/s 
     |████████████████████████████████| 85 kB 1.2 MB/s 
     |████████████████████████████████| 88 kB 3.9 MB/s 
     |████████████████████████████████| 5.2 MB 43.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 662 kB 50.5 MB/s 
     |████████████████████████████████| 5.8 MB 41.8 MB/s 
     |████████████████████████████████| 1.3 MB 45.0 MB/s 
     |████████████████████████████████| 182 kB 53.6 MB/s 
     |████████████████████████████████| 7.6 MB 49.5 MB/s 
     |████████████████████████████████| 1.1 MB 36.2 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700873 sha256=3e94ad39c76bd0028576977e992cf6f5130ab8b9808551ae926c0c8661d3a97f
  Stored in directory: /root/.cache/

In [2]:
!pip install pyldavis
!pip install gensim pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=2785dad6c98c1a2d18bf65bac2cf14c47f5121470c7c37e642cae1b4ca34ccb3
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=d47403aa161d6f5b229df73a99c745768dcdc4ea64c39ca4fabcb59fc52409e8
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install octis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#import dependencies 
import pandas as pd
import numpy as np
import string
#import spacy
import nltk
import os
import re
import gensim
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim import corpora
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models
from collections import defaultdict
import time 
from gensim.models import Phrases
from gensim.models import CoherenceModel
from gensim.models import LdaModel, LdaMulticore
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

#from octis.models.LDA import LDA
from sentence_transformers import SentenceTransformer
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
#from octis.evaluation_metrics.coherence_metrics import Coherence

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#load reduced training data 
yahoo_ans = pd.read_csv('/content/yahoo_cleaned_tr.csv')
yahoo_ans

In [8]:
doc = list(yahoo_ans['Text'])
targets = list( yahoo_ans['Class'])
target_names = yahoo_ans['Class_Name']
classes = [yahoo_ans['Class_Name'][i] for i in yahoo_ans['Class']]

In [9]:
category_names = ['Society_Culture','Science_Mathematics', 
                  'Health','Education _Reference', 
                  'Computers_Internet', 'Sports',
                  'Business_Finance', 'Entertainment_Music', 
                   'Family_Relationships', 'Politics_Government']

categories = yahoo_ans['Class']

In [10]:
#labels to add
labels_to_add = ['Health', 'Science_Mathematics',
              'Entertainment_Music', 'Politics_Government',
              'Society_Culture',]
indices = [category_names.index(label) for label in labels_to_add]
y = [label if label in indices else -1 for label in categories]

In [11]:
umap_model = UMAP(n_neighbors=10, n_components=20, 
                  min_dist=0.0, metric='cosine', random_state=0)

In [12]:
vectorizer_model = CountVectorizer(stop_words="english")

In [13]:
bertopic_model = BERTopic(umap_model=umap_model, 
                          verbose=True, 
                          nr_topics=10, 
                          n_gram_range=(1, 2), 
                          top_n_words=20 ,
                          vectorizer_model=vectorizer_model,
                          calculate_probabilities=True,
                          language="English"
                            )

In [14]:
%%time
#Run BERTopic model
topics, probabilities  = bertopic_model.fit_transform(doc, y=y)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

2022-12-05 18:30:46,114 - BERTopic - Transformed documents to Embeddings
2022-12-05 18:34:12,962 - BERTopic - Reduced dimensionality
2022-12-05 18:46:28,326 - BERTopic - Clustered reduced embeddings
2022-12-05 18:47:25,020 - BERTopic - Reduced number of topics from 523 to 11


CPU times: user 50min 43s, sys: 49.5 s, total: 51min 33s
Wall time: 51min 24s


In [15]:

#Get list of topics
bertopic_model.get_topic_info()

,Topic,Count,Name
0,-1,39447,-1_like_know_people_time
1,0,1717,0_love_like_friend_guy
2,1,1505,1_song_music_ni_band
3,2,1476,2_bush_war_president_iraq
4,3,1142,3_weight_fat_eat_diet
5,4,898,4_number_10_equation_12
6,5,822,5_cup_team_world_game
7,6,786,6_computer_drive_window_virus
8,7,774,7_yahoo_mail_email_account
9,8,744,8_team_game_baseball_year


#PLots

In [39]:
bertopic_model.visualize_topics()

In [40]:
bertopic_model.visualize_distribution(probabilities[3])

In [41]:
bertopic_model.visualize_barchart(top_n_topics =  10 )

In [42]:
topics_per_class = bertopic_model.topics_per_class(doc,  classes=classes)
fig_semi_supervised = bertopic_model.visualize_topics_per_class(topics_per_class,top_n_topics=10, width=900)
fig_semi_supervised

7it [00:04,  1.55it/s]


In [43]:
bertopic_model.visualize_hierarchy()

/usr/local/lib/python3.8/dist-packages/plotly/figure_factory/_dendrogram.py:350: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.8/dist-packages/plotly/figure_factory/_dendrogram.py:351: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.8/dist-packages/plotly/figure_factory/_dendrogram.py:352: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.8/dist-packages/plotly/figure_factory/_dendrogram.py:353: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead



In [44]:
bertopic_model.visualize_distribution(probabilities[9])

In [45]:
#Visualize Topic Similarity
bertopic_model.visualize_heatmap()

/usr/local/lib/python3.8/dist-packages/plotly/express/_imshow.py:354: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [46]:
bertopic_model.visualize_term_rank(log_scale=True)

# Coherence Scores

In [30]:
# Preprocess Documents
documents = pd.DataFrame({"Document": doc,
                          "ID": range(len(doc)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = bertopic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = bertopic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in bertopic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

In [17]:
# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

0.812147751730121

In [18]:
# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence = coherence_model.get_coherence()
coherence

-0.23786174843447822

# Topic diversity

In [36]:
output_tm = {"topics": topic_words}

In [38]:
metric = TopicDiversity(topk=10) # Initialize metric
topic_diversity_score = metric.score(output_tm) # Compute score of the metric
topic_diversity_score

0.92